Project Description

Objective: Use the GEDI L2B product to classify the San Pedro Riparian National Conservation Area (SPRNCA) by Foliage Height Diversity (FHD). 

FHD is an important metric for estimating ecological diversity and habitat suitability. Bird species are particularly sensitive in their habitat selection to vertical structure characteristics and the SPRNCA is an internationally recognized habitat zone for migratory and wintering birds. Zones would be classified using unsupervised techniques such as Agglomerative Clustering or DBSCAN. Once regions are classified,  I can evaluate present day trends in water stress over the different FHD classes to evaluate if parts of the corridor that are well suited for sensitive species habitat are particularly susceptible to water stress. To measure water stress, I will use either the ECOSTRESS Evaporative Stress Index (ESI) or EEflux’s METRIC based ET product coupled with ERA5’s PET estimate, depending on data availability. The primary outputs of this project will be maps classifying SPRNCA by FHD and time series plots and maps of the ESI for each FHD class.

The GEDI L2B product has an average spatial footprint of 25 meters and there are ~ 220 observations covering the SPRNCA corridor (I still need to determine how well this time series covers the SPRNCA). So this analysis would be limited to portions of the SPRNCA that have a riparian corridor wider than 25 meters. I also have near daily cloud masks from this area derived from Planet Labs imagery that I will use to filter the L2B and ESI products.

Notes

GEDI data come in groups, and xarray can only read one at a time.

Quality FAQ with suggestions are here: https://lpdaac.usgs.gov/documents/589/GEDIL02_User_Guide_V1.pdf

conda install pyinterp -c conda-forge possible option for interpolation as it supports unstructured grids, 3D

xarray question: https://stackoverflow.com/questions/60161815/define-coordinates-meant-to-work-with-particular-variables-in-xarray

In [1]:
import xarray as xr
import rioxarray as rx
import numpy as np
from pathlib import Path

In [2]:
beam_groups = ["/BEAM0000", "/BEAM1011", "/BEAM1000", "/BEAM0110", "/BEAM0101", "/BEAM0011", "/BEAM0010", "/BEAM0001"]
beam_sub_groups = ["ancillary", "geolocation", "land_cover_data", "rx_processing"]
meta_group = "METADATA/DatasetIdentification"

In [3]:
def open_beams(file_path, beam_groups = ["/BEAM0000", "/BEAM1011", "/BEAM1000", "/BEAM0110", "/BEAM0101", "/BEAM0011", "/BEAM0010", "/BEAM0001"]):
    datasets = []
    for beam in beam_groups:
        print(beam)
        # coord related values that need to be assigned to the dataset.
        ds_geo = xr.open_dataset(file_path, group=f"/{beam}/geolocation/")
        lat_coords = ds_geo.lat_highestreturn.values
        lon_coords = ds_geo.lon_highestreturn.values
        time_coords = ds_geo.delta_time.values
        
        # renaming and prooer meter units for z, vertical profile
        dz = xr.open_dataset(file_path, group=f"/{beam}/ancillary").dz.values
        ds = xr.open_dataset(file_path, group=beam)
        dim_names = list(ds.dims)
        ds = ds.rename({dim_names[1]:"z"}).drop_dims(dim_names[2]) # phony 6 only used for directional gap prob profile. 
        ds['z'] = ds.z * dz # make z have units in meters
        
        # assigning location and time coords to the variables depending on if they represent avg over profile or are taken along profile
        ds = ds.rename({dim_names[0]:"y"})
        ds['y'] = lat_coords
        ds = ds.assign({"x":lon_coords, "time":time_coords})
        
        datasets.append(ds)
    return datasets

In [4]:
file_paths = list(Path("data").glob("*.h5"))

In [16]:
list(xr.open_dataset(file_paths[0], group="/BEAM1011").dims)

['phony_dim_53', 'phony_dim_54', 'phony_dim_55']

In [89]:
test_beam.sensitivity
test_beam.fhd_normal
test_beam.l2a_quality_flag
test_beam.l2b_quality_flag
test_beam.pai
test_beam.pai_z
test_beam.pavd_z
test_beam.l2b_quality_flag

<xarray.Dataset>
Dimensions:                 (phony_dim_4: 475621, phony_dim_5: 30, phony_dim_6: 8944126)
Dimensions without coordinates: phony_dim_4, phony_dim_5, phony_dim_6
Data variables:
    algorithmrun_flag       (phony_dim_4) uint8 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0
    beam                    (phony_dim_4) uint16 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0
    channel                 (phony_dim_4) uint8 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0
    cover                   (phony_dim_4) float32 nan nan nan ... nan nan nan
    cover_z                 (phony_dim_4, phony_dim_5) float32 nan nan ... nan
    delta_time              (phony_dim_4) datetime64[ns] 2019-06-12T14:11:24.600074800 ... 2019-06-12T15:26:19.994699152
    fhd_normal              (phony_dim_4) float32 nan nan nan ... nan nan nan
    l2a_quality_flag        (phony_dim_4) uint8 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0
    l2b_quality_flag        (phony_dim_4) uint8 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0
    master_frac             (phony_dim_4) timedelta64[ns] 00:00:00.600074 ... 00:00:00.994699
    master_int              (phony_dim_4) timedelta64[ns] 527 days 14:11:24 ... 527 days 15:26:19
    num_detectedmodes       (phony_dim_4) uint8 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0
    omega                   (phony_dim_4) float32 1.0 1.0 1.0 ... 1.0 1.0 1.0
    pai                     (phony_dim_4) float32 nan nan nan ... nan nan nan
    pai_z                   (phony_dim_4, phony_dim_5) float32 nan nan ... nan
    pavd_z                  (phony_dim_4, phony_dim_5) float32 nan nan ... nan
    pgap_theta              (phony_dim_4) float32 nan nan nan ... nan nan nan
    pgap_theta_error        (phony_dim_4) float32 nan nan nan ... nan nan nan
    pgap_theta_z            (phony_dim_6) float32 nan nan nan ... nan nan nan
    rg                      (phony_dim_4) float32 nan nan nan ... nan nan nan
    rh100                   (phony_dim_4) int16 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0
    rhog                    (phony_dim_4) float32 0.4 0.4 0.4 ... 0.4 0.4 0.4
    rhog_error              (phony_dim_4) float32 nan nan nan ... nan nan nan
    rhov                    (phony_dim_4) float32 0.6 0.6 0.6 ... 0.6 0.6 0.6
    rhov_error              (phony_dim_4) float32 nan nan nan ... nan nan nan
    rossg                   (phony_dim_4) float32 0.5 0.5 0.5 ... 0.5 0.5 0.5
    rv                      (phony_dim_4) float32 nan nan nan ... nan nan nan
    rx_range_highestreturn  (phony_dim_4) float64 4.147e+05 ... 4.195e+05
    rx_sample_count         (phony_dim_4) int64 4 4 4 4 4 4 4 ... 4 4 4 4 4 4 4
    rx_sample_start_index   (phony_dim_4) int64 1 5 9 ... 8944119 8944123
    selected_l2a_algorithm  (phony_dim_4) uint8 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1
    selected_rg_algorithm   (phony_dim_4) uint8 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0
    sensitivity             (phony_dim_4) float32 -23.674726 ... -2.002046
    shot_number             (phony_dim_4) uint64 28220002300000001 ... 28220011900475621
    stale_return_flag       (phony_dim_4) uint8 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1
    surface_flag            (phony_dim_4) uint8 1 1 1 1 1 1 1 ... 0 0 0 0 0 0 0
Attributes:
    wp-l2-l2b_githash:  344b44a3adc93983cc84ade093020500f87fee00
    wp-l2-l2b_version:  20191201.0.0
    description:        Coverage beam

In [154]:
ds = xr.Dataset({'pai_z': (['x', 'y', 'time', 'z'],  test_beam.pai_z),
                 coords={'x': (['x', 'y'], lon_coords),
                         'y': (['x', 'y'], lat_coords),
                         'time': time_coords,
                        "z": test_beam.phony_dim_5.values}})

SyntaxError: invalid syntax (<ipython-input-154-04dc661b0266>, line 2)